In [1]:
import operator
from typing import List, Tuple, Annotated, TypedDict, Union, Literal
from langchain import hub
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOllama
from langchain.tools import tool
from duckduckgo_search import DDGS
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.prebuilt import create_react_agent
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph

In [2]:
#tools
#wrapper = DuckDuckGoSearchAPIWrapper(max_results=20)
#web_search_tool = DuckDuckGoSearchRun(api_wrapper=wrapper)
#web_search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)

@tool("search_duckduckgo", return_direct=True)
def search_duckduckgo(query: str, num_results: int = 5, content: bool = True):
    ''' Provides search results from duckduckgo '''
    results = DDGS().text(query, max_results=num_results)
    if content:
        formatted_results = [{"url": result['href'], "content": result['body']} for result in results]
    else:
        formatted_results = [{"url": result['href']} for result in results]
    return formatted_results


tools = [search_duckduckgo]

# Prompt style
prompt = hub.pull("wfh/react-agent-executor")
prompt.pretty_print()

# Creating an Agent Executor with tools
groq_api_key = "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"
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-70b-8192")
#llm = ChatOllama(model = "llama3", temperature=0)
agent_executor = create_react_agent(llm, tools, messages_modifier=prompt)

================================ System Message ================================

You are a helpful assistant.

============================= Messages Placeholder =============================

{{messages}}


In [3]:
agent_executor.invoke({"messages": [("user", "Give me a list of G20 Countries")]})

{'messages': [HumanMessage(content='Give me a list of G20 Countries', id='3e9a53b3-96a6-42bd-8372-35c8679c7bb0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6bv2', 'function': {'arguments': '{"query":"list of G20 countries"}', 'name': 'search_duckduckgo'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 966, 'total_tokens': 1015, 'completion_time': 0.146656535, 'prompt_time': 0.52503276, 'queue_time': None, 'total_time': 0.671689295}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-965dd3d5-4103-47fe-ba9e-7d0c5e73f18d-0', tool_calls=[{'name': 'search_duckduckgo', 'args': {'query': 'list of G20 countries'}, 'id': 'call_6bv2'}]),
  ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/G20", "content": "The G20 or Group of 20 is an intergovernmental forum comprising 19 sovereign countries, the European Union (EU), and the 

In [4]:
# Defining the State
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [5]:
# Defining the planning step
class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps. Break down
the given objective into simple tasks. For example: If i ask for the hometown of the current us open winner, the plan would be to 1. Find who the current
winner of US open, once you have the name 2. Find the home town""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-70b-8192").with_structured_output(Plan)

planner.invoke(
    {
        "messages": [
            ("user", "Compile table of electricity consumption per capita for g20 nations")
        ]
    }
)

C:\Users\safim\anaconda3\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The method `ChatGroq.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Plan(steps=['Find the list of G20 nations', 'Find the electricity consumption per capita for each nation', 'Compile the data into a table'])

In [6]:
# Defining the re-planing step
class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.Break down
the given objective into simple tasks. For example: If i ask for the hometown of the current us open winner, the plan would be to 1. Find who the current
winner of US open, once you have the name 2. Find the home town

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)

replanner = replanner_prompt | ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-70b-8192").with_structured_output(Act)

In [7]:
# Create the Graph
async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""For the following plan:
{plan_str}\n\nYou are tasked with executing step. You only have one tool: search_duckduckgo, use it when needed {1}, {task}."""
    agent_response = await agent_executor.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": (task, agent_response["messages"][-1].content),
    }

async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute) -> Literal["agent", "__end__"]:
    if "response" in state and state["response"]:
        return "__end__"
    else:
        return "agent"

In [8]:
workflow = StateGraph(PlanExecute)

# Adding the plan node
workflow.add_node("planner", plan_step)

# Adding the execution step
workflow.add_node("agent", execute_step)

# Adding a replan node
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

# From plan to agent
workflow.add_edge("planner", "agent")

# From agent, to replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Pass in the function that will determine which node is called next.
    should_end,
)

app = workflow.compile()

In [9]:
config = {"recursion_limit": 50}
inputs = {"input": "Compile table of electricity consumption per capita for g20 nations"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ['Find the list of G20 nations', 'Find the electricity consumption per capita for each nation', 'Compile the data into a table']}
{'past_steps': ('Find the list of G20 nations', 'Now that we have the list of G20 nations and the information on electricity consumption per capita, we can compile the data into a table. Here is the compiled data:\n\n| Country | Electricity Consumption per Capita (kWh) |\n| --- | --- |\n| Argentina | 2,531 |\n| Australia | 9,495 |\n| Brasil | 2,455 |\n| Canada | 14,764 |\n| China | 4,454 |\n| France | 7,374 |\n| Germany | 7,035 |\n| India | 818 |\n| Indonesia | 744 |\n| Italy | 5,534 |\n| Japan | 7,374 |\n| Korea | 10,162 |\n| Mexico | 2,144 |\n| Russia | 6,438 |\n| Saudi Arabia | 9,295 |\n| South Africa | 4,144 |\n| T\\u00fcrkiye | 2,844 |\n| UK | 5,538 |\n| USA | 12,154 |\n\nNote: The electricity consumption per capita values are approximate and based on the data from the provided sources.')}
{'response': 'The table of electricity consumption per 

Object `agent_executor.with_structured_output` not found.
